In [1]:
data_list = "/data/cbw/lane-detection-pytorch/datasets/vil100/data/train.txt"

with open(data_list, "r") as f:
    img_infos = [
        img_name.strip()[1:] for img_name in f.readlines()
    ]
annotations = [
    img_path.replace("JPEGImages", "Json") + ".json" for img_path in img_infos
]
mask_paths = [
    img_path[:-3].replace("JPEGImages", "Annotations") + "png" for img_path in img_infos
]

print(len(img_infos), len(annotations), len(mask_paths))
print(img_infos[0:3])
print(annotations[0:3])
print(mask_paths[0:3])

from pathlib import Path
# help(Path)
test_pred_anno = Path("tmp").joinpath(img_infos[0].replace("JPEGImages", "Json").replace(".jpg", ".jpg.json"))
print(test_pred_anno)

8000 8000 8000
['JPEGImages/0_Road001_Trim003_frames/00246.jpg', 'JPEGImages/0_Road001_Trim003_frames/00075.jpg', 'JPEGImages/0_Road001_Trim003_frames/00078.jpg']
['Json/0_Road001_Trim003_frames/00246.jpg.json', 'Json/0_Road001_Trim003_frames/00075.jpg.json', 'Json/0_Road001_Trim003_frames/00078.jpg.json']
['Annotations/0_Road001_Trim003_frames/00246.png', 'Annotations/0_Road001_Trim003_frames/00075.png', 'Annotations/0_Road001_Trim003_frames/00078.png']
tmp/Json/0_Road001_Trim003_frames/00246.jpg.json


In [2]:
import os 
time_window_size = 3
def get_img_clip(img_path):
    clip_dir = os.path.dirname(img_path)
    frame_name = os.path.basename(img_path)
    frame_number = int(frame_name.split(".")[0])
    return clip_dir, frame_number

clips = {}
for anno_idx, anno in enumerate(annotations):
    img_path = os.path.join("/home/seasoning/lane/datasets/VIL100", anno)
    img_clip, frame_idx = get_img_clip(img_path)
    if img_clip not in clips:
        clips[img_clip] = [(frame_idx, anno_idx)]
    else:
        clips[img_clip].append((frame_idx, anno_idx))
sequences_idxs = []
for clip in clips:
    clip_data = sorted(clips[clip], key=lambda x: x[0])
    clip_anno_idxs = [item[1] for item in clip_data]
    for i in range(time_window_size - 1):
        sequences_idxs.append([clip_anno_idxs[i]] * time_window_size)
    for i in range(len(clip_anno_idxs) - time_window_size + 1):
        sequences_idxs.append(clip_anno_idxs[i : i + time_window_size])
sequences_idxs[0:5]


[[99, 99, 99], [89, 89, 89], [99, 89, 56], [89, 56, 15], [56, 15, 22]]

In [ ]:
anno_dir = "/home/seasoning/lane/datasets/culane/driver_23_30frame/05151640_0419.MP4/00000.lines.txt"
shapes = []
with open(anno_dir, "r") as anno_f:
    lines = anno_f.readlines()
    for line in lines:
        coords = []
        coords_str = line.strip().split(" ")
        for i in range(len(coords_str) // 2):
            coord_x = float(coords_str[2 * i])
            coord_y = float(coords_str[2 * i + 1])
            coords.append(coord_x)
            coords.append(coord_y)
        if len(coords) > 3:
            shapes.append(coords)
print(shapes)

In [ ]:
import json
anno_dir = "/home/seasoning/lane/datasets/VIL100/Json/0_Road001_Trim003_frames/00000.jpg.json"
with open(anno_dir, "r") as anno_f:
    lanes = [
                lane["points"] for lane in json.load(anno_f)["annotations"]["lane"]
            ]
print(lanes)
print(len(lanes))
lanes = [[[point[0], point[1] - 1] for point in lane] for lane in lanes]
lanes = [[item for point in lane for item in point] for lane in lanes]
print(lanes)
# lanes_1 = [[item for point in lane for item in [point[0], point[1] - 1]] for lane in lanes]
# print(lanes_1)  



In [ ]:
anno_dir = "/home/seasoning/lane/datasets/culane/driver_23_30frame/05151640_0419.MP4/00000.lines.txt"
shapes = []
with open(anno_dir, "r") as anno_f:
    lines = anno_f.readlines()
    for line in lines:
        coords = []
        coords_str = line.strip().split(" ")
        for i in range(len(coords_str) // 2):
            coord_x = float(coords_str[2 * i])
            coord_y = float(coords_str[2 * i + 1])
            coords.append(coord_x)
            coords.append(coord_y)
        if len(coords) > 3:
            shapes.append(coords)
print(shapes)

In [ ]:
# test_pipline_dataset
import os
import sys
sys.path.append("/home/seasoning/VOD/CLRerNet")
from libs.datasets import VIL100Dataset, CulaneDataset

data_root = "/home/seasoning/lane/datasets/VIL100"
data_list = data_root + "/data/train.txt"
img_scale = (800, 320)
compose_cfg = dict(bboxes=False, keypoints=True, masks=False)
img_norm_cfg = dict(
    mean=[0.0, 0.0, 0.0], std=[255.0, 255.0, 255.0], to_rgb=False
)
val_al_pipeline = [
    dict(type="Compose", params=compose_cfg),
    dict(type="Resize", height=img_scale[1], width=img_scale[0], p=1),
]
val_pipeline = [
    dict(type="albumentation", pipelines=val_al_pipeline, cut_unsorted=False),
    dict(type="Normalize", **img_norm_cfg),
    dict(type="DefaultFormatBundle"),
    dict(
        type="CollectCLRNet",
        max_lanes=6,
        keys=["img"],
        meta_keys=[
            "filename",
            "sub_img_name",
            "ori_shape",
            "eval_shape",
            "img_shape",
            "img_norm_cfg",
            "ori_shape",
            "img_shape",
            "gt_points",
            "gt_masks",
            "lanes",
        ],
    ),
]
dataset = VIL100Dataset(data_root, data_list, val_pipeline, test_mode=False)
dataset_DC = dataset.prepare_train_img(0)
print(dataset)

data_root_culane = "/home/seasoning/VOD/CLRerNet/dataset/culane"
data_list_culane = data_root_culane + "/list/train_gt.txt"
diff_file = data_root_culane + "/list/train_diffs.npz"
crop_bbox = [0, 270, 1640, 590]
val_al_pipeline_culane = [
    dict(type="Compose", params=compose_cfg),
    dict(
        type="Crop",
        x_min=crop_bbox[0],
        x_max=crop_bbox[2],
        y_min=crop_bbox[1],
        y_max=crop_bbox[3],
        p=1,
    ),
    dict(type="Resize", height=img_scale[1], width=img_scale[0], p=1),
]
train_pipeline_culane = [
    dict(
        type="albumentation", pipelines=val_al_pipeline_culane, cut_unsorted=False
    ),
    dict(type="Normalize", **img_norm_cfg),
    dict(type="DefaultFormatBundle"),
    dict(
        type="CollectCLRNet",
        keys=["img"],
        meta_keys=[
            "filename",
            "sub_img_name",
            "ori_shape",
            "img_shape",
            "img_norm_cfg",
            "ori_shape",
            "eval_shape",
            "img_shape",
            "gt_points",
            "gt_masks",
            "lanes",
        ],
    ),
]
dataset_culane = CulaneDataset(data_root_culane, data_list_culane, train_pipeline_culane, test_mode=False)
# dataset_DC = dataset_culane.prepare_train_img(0)

import cv2
import numpy as np
from scipy import interpolate
img = dataset_DC["img"]._data*255
img = img.numpy().transpose(1, 2, 0)
cv2.imwrite("img.jpg", img)

from libs.utils.visualizer import draw_lane
img_show = img.copy()
lanes_xs = dataset_DC["img_metas"]._data["lanes"][:,6:]
py_coord = 320 - np.float32(np.round(np.linspace(319, 0 + 1, 319)))
# print(py_coord)
offset_ys = np.arange(320, -1, -320/71)
lanes_ys = offset_ys.reshape(1,72).repeat(lanes_xs.shape[0], 0)
lanes = np.concatenate((lanes_xs.reshape(-1,72, 1), lanes_ys.reshape(-1,72, 1)), axis=2)
print(offset_ys)
for i in range(len(lanes)):
    # print(lanes[i])
    lane_pts = np.array(lanes[i], np.int32).reshape(-1, 2)
    # print("lane reshape", lane_pts)
    f = interpolate.splrep(lane_pts[:, 1][::-1], lane_pts[:, 0][::-1], k=1, s=20)
    new_x_pts = interpolate.splev(offset_ys, f)
    new_lane_pts = np.concatenate((new_x_pts.reshape(-1, 1), offset_ys.reshape(-1, 1)), axis=1)
    print("new_lane_pts_culane", new_lane_pts)
    draw_lane(lane_pts, img_show, img_show.shape, width=3, color=(255, 0, 0))
cv2.imwrite("img_show_vil100.jpg", img_show)

In [ ]:
import sys
sys.path.append("/home/seasoning/VOD/CLRerNet")
from libs.datasets import VIL100Dataset, CulaneDataset

def vis_interp_extrap(
        data_root, data_list, pipeline=[], dataset_obj=VIL100Dataset, diff_file=None, 
        crop_bbox=None, img_scale=(800, 320), img_norm_cfg=None,
        compose_cfg = dict(bboxes=False, keypoints=True, masks=False)
):
    dataset = dataset_obj(data_root, data_list, pipeline, test_mode=False)
    for i in np.arange(5210,6000,66):
        dataset_DC = dataset.prepare_train_img(i)
        img_i = dataset_DC["img"]._data*255
        img_i = img_i.numpy().transpose(1, 2, 0)
        cv2.imwrite(f"img_{i}.jpg", img_i)
        img_i_show = img_i.copy()
        lanes_i = dataset_DC["img_metas"]._data["gt_points"]
        offset_ys = np.arange(320, -1, -320/71)
        for i in range(len(lanes_i)):
            lane_pts = np.array(lanes_i[i], np.int32).reshape(-1, 2)
            # f = interpolate.splrep(lane_pts[:, 1][::-1], lane_pts[:, 0][::-1], k=1, s=20)
            # new_x_pts = interpolate.splev(offset_ys, f)
            # new_lane_pts = np.concatenate((new_x_pts.reshape(-1, 1), offset_ys.reshape(-1, 1)), axis=1)
            draw_lane(lane_pts, img_i_show, img_i_show.shape, width=3, color=(255, 0, 0))
        cv2.imwrite(f"img_{i}_show.jpg", img_i_show)

vis_interp_extrap(data_root, data_list, val_pipeline, VIL100Dataset)


In [ ]:
import numpy as np
from scipy.interpolate import *
import matplotlib.pyplot as plt

points = [[0, 0], [4, 4], [-1, 9], [-4, -1], [-1, -9], [4, -4], [0, 0]]
points = np.asarray(points)

n = 50 
ts = np.linspace(0, 1, len(points))
ts_new = np.linspace(0, 1, n)

(t0_0,c0_0,k0_0), u = splprep(points[:,[0]].T, s=0, k=3)
(t0_1,c0_1,k0_1), u = splprep(points[:,[1]].T, s=0, k=3)
p0_new = np.r_[np.asarray(splev(ts_new, (t0_0,c0_0,k0_0))),
               np.asarray(splev(ts_new, (t0_1,c0_1,k0_1))),
                ].T

# splprep/splev
(t1,c1,k1), u = splprep(points.T, s=0, k=3)
p1_new = splev(ts_new, (t1,c1,k1))
# BSpline from splprep
p2_new = BSpline(t1, np.asarray(c1).T, k=k1)(ts_new)
# splrep/splev (per dimension)
(t3_0,c3_0,k3_0) = splrep(ts, points[:,0].T, s=0, k=3)
(t3_1,c3_1,k3_1) = splrep(ts, points[:,1].T, s=0, k=3)
p3_new = np.c_[splev(ts_new, (t3_0,c3_0,k3_0)),
               splev(ts_new, (t3_1,c3_1,k3_1)),
               ]
# Bspline from splrep
p4_new = np.c_[BSpline(t3_0, np.asarray(c3_0), k=k3_0)(ts_new),
               BSpline(t3_1, np.asarray(c3_1), k=k3_1)(ts_new),
               ]
# UnivariateSpline
p5_new = np.c_[UnivariateSpline(ts, points[:,0], s=0, k=3)(ts_new),
               UnivariateSpline(ts, points[:,1], s=0, k=3)(ts_new),]
# make_interp_spline
p6_new = make_interp_spline(ts, points, k=3)(ts_new)
# CubicSpline
p7_new = CubicSpline(ts, points, bc_type="clamped")(ts_new)
# interp1d
p8_new = interp1d(ts, points.T, kind="cubic")(ts_new).T

fig, ax = plt.subplots()
ax.plot(*points.T, "o-", label="Original points")
ax.plot(*p1_new,   "o-", label="1: splprep/splev")
ax.plot(*p2_new.T, "x-", label="1: BSpline from splprep")
ax.plot(*p3_new.T, "o-", label="2: splrep/splev")
ax.plot(*p4_new.T, "x-", label="2: BSpline from splrep")
ax.plot(*p5_new.T, "*-", label="2: UnivariateSpline")
ax.plot(*p6_new.T, "+-", label="2: make_interp_spline")
ax.plot(*p7_new.T, "x-", label="3: CubicSpline")
#ax.plot(*p8_new.T, "k+-", label="3: interp1d")
#ax.plot(*p0_new.T, "k+-", label="3: CubicSpline")
ax.set_aspect("equal")
ax.grid("on")
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
plt.show()